In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [42]:
def load_video(path, ms=100):
    
    # Video Capture 객체 생성
    capture = cv2.VideoCapture(path)
    
    # Frame을 저장할 List 선언
    frames = []
    
    while capture.isOpened(): # Video Capture가 준비되었는지 확인
        
        run, frame = capture.read() # 다음 Frame 읽기
        # ROI 설정 (x, y, width, height)
        roi_x, roi_y, roi_w, roi_h = 0, 280, 852, 200
        
        # ROI 추출
        roi = frame[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w]

        # 그레이스케일 변환
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        # 원본 프레임에 엣지 결과를 덮어쓰기 (ROI 영역만)
        frame[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w] = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        
        
        if run: # Frame을 읽은 경우
            # 결과 출력
            cv2.imshow('Frame with ROI Edges', frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else: # 재생이 완료되어 더 이상 Frame을 읽을 수 없는 경우
            break
        
        # Frame List에 추가
        frames.append(frame)

    capture.release() # Capture 자원 반납
    cv2.destroyAllWindows() # 창 제거
    
    return np.array(frames, dtype='uint8')

In [43]:
# Video가 저장된 경로 입력
PATH = r".\plank_sample2.mp4"
# PATH = r"C:\Users\admin\Desktop\KUDIP\Video Samples\Hand Video2.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\highway.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\earth.avi"

# Video 재생 및 반환 (Numpy Array)
video = load_video(PATH)

In [7]:
print(f"- Data Type: {type(video)}")
print(f"- Data Shape: {video.shape} *Frames x Height x Width x Channel")
print(f"- Maximum Intensity: {video.max()}")
print(f"- Minimum Intensity: {video.min()}")

- Data Type: <class 'numpy.ndarray'>
- Data Shape: (201, 480, 852, 3) *Frames x Height x Width x Channel
- Maximum Intensity: 255
- Minimum Intensity: 0


In [15]:
def find_contours(morphed):
    contours, _ = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def draw_keypoints(image, keypoints):
    for point in keypoints:
        cv2.circle(image, point, 5, (0, 0, 255), -1)
    return image

def detect_hip_position(contours):
    keypoints = []
    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # 작은 윤곽선 무시
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                keypoints.append((cX, cY))
    
    if len(keypoints) > 0:
        keypoints = sorted(keypoints, key=lambda k: k[1])  # y좌표 기준 정렬
        hip = keypoints[len(keypoints) // 2]  # 중간 값이 엉덩이
        return hip
    return None


In [18]:
import cv2
import numpy as np

# 영상 읽기
cap = cv2.VideoCapture('plank_sample2.mp4')

# 타이머 설정
total_time = 0
start_time = cv2.getTickCount()
stop_timer = False

bg_subtractor = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, c = frame.shape
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.medianBlur(gray, 3)
    fg_mask = bg_subtractor.apply(blurred)
    _, binary = cv2.threshold(fg_mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # morphology 연산
    kernel = np.ones((5, 5), np.uint8)
    morphed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)
    morphed = cv2.morphologyEx(morphed, cv2.MORPH_OPEN, kernel, iterations=2)
    
    contours = find_contours(morphed)
    hip_position = detect_hip_position(contours)
    if hip_position:
        print(hip_position)
    # Canny 엣지 검출
    # edges = cv2.Canny(blurred, 50, 150)
    

    cv2.imshow('Plank Pose Monitoring', binary[:, w*2//5:w*3//5])

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


(429, 172)
(261, 351)
(353, 361)
(352, 360)
(266, 367)
(552, 409)
(554, 402)
(567, 394)
(576, 393)
(585, 391)
(248, 396)
(481, 406)
(610, 403)
(610, 403)
(611, 407)
(248, 409)
(248, 408)
(609, 419)
(609, 420)
(610, 408)
(609, 408)
(610, 409)
(615, 425)
(549, 425)
(518, 425)
(518, 424)
(510, 418)
(506, 418)
(596, 395)
(596, 394)
(520, 414)
(569, 387)
(465, 415)
(220, 392)
(219, 386)
(569, 381)
(569, 381)
(217, 385)
(614, 411)
(615, 415)
(615, 415)
(613, 409)
(618, 407)
(606, 410)
(606, 410)
(605, 408)
(608, 406)
(613, 422)
(613, 432)
(613, 432)
(613, 431)
(613, 428)
(614, 431)
(614, 428)
(378, 359)
(378, 358)
(375, 358)
(374, 358)
(616, 430)
(616, 430)
(616, 430)
(616, 430)
(618, 428)
(618, 424)
(619, 415)
(623, 413)
(625, 414)
(393, 339)
(223, 364)
(223, 362)
(223, 363)
(223, 363)
(224, 362)
(223, 363)
(223, 363)
(223, 363)
(224, 362)
(224, 363)
(225, 363)
(225, 363)
(222, 365)
(222, 365)
(229, 361)
(230, 362)
(224, 363)
(223, 363)
(222, 360)
(222, 360)
(222, 359)
(228, 361)
(228, 361)